# Simple NIfTI Viewer
Interactive viewer for `.nii.gz` files with scrollable slices and adjustable windowing.

In [7]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

class SimpleNiftiViewer:
    def __init__(self):
        self.img_vol = None
        
        # UI Elements
        self.img_path_text = widgets.Text(description="Image Path:", placeholder="path/to/image.nii.gz", layout=widgets.Layout(width='70%'))
        self.load_button = widgets.Button(description="Load Volume", button_style='primary')
        
        self.slice_slider = widgets.IntSlider(description="Slice:", continuous_update=True, layout=widgets.Layout(width='50%'))
        self.axis_dropdown = widgets.Dropdown(description="View:", options=[("Axial", 2), ("Sagittal", 0), ("Coronal", 1)], value=2)
        
        self.window_dropdown = widgets.Dropdown(
            description="Window:",
            options=[
                ("Full Range", "full"),
                ("CT Bone (min:-500, max:1300)", (-500, 1300)),
                ("CT Soft Tissue (min:-150, max:350)", (-150, 350)),
                ("CT Lung (min:-1000, max:400)", (-1000, 400)),
                ("Custom", "custom"),
            ],
            value="full"
        )
        self.vmin_input = widgets.FloatText(value=-1024, description="vmin:", layout=widgets.Layout(width='150px'))
        self.vmax_input = widgets.FloatText(value=1024, description="vmax:", layout=widgets.Layout(width='150px'))
        self.custom_box = widgets.HBox([self.vmin_input, self.vmax_input], layout=widgets.Layout(visibility='hidden'))
        
        self.output = widgets.Output()
        
        # Event Handlers
        self.load_button.on_click(self.load_data)
        self.slice_slider.observe(self.update_plot, names='value')
        self.axis_dropdown.observe(self.on_axis_change, names='value')
        self.window_dropdown.observe(self.on_window_change, names='value')
        self.vmin_input.observe(self.update_plot, names='value')
        self.vmax_input.observe(self.update_plot, names='value')
        
        # Layout
        self.controls = widgets.VBox([
            widgets.HBox([self.img_path_text, self.load_button]),
            widgets.HBox([self.slice_slider, self.axis_dropdown]),
            widgets.HBox([self.window_dropdown, self.custom_box])
        ])
        
        display(self.controls, self.output)

    def load_data(self, b):
        with self.output:
            img_path = self.img_path_text.value
            if not os.path.exists(img_path):
                print(f"Error: Image path '{img_path}' does not exist.")
                return
            
            print(f"Loading {img_path}...")
            self.img_vol = nib.load(img_path).get_fdata()
            
            if self.window_dropdown.value == "full":
                self.vmin_input.value = np.min(self.img_vol)
                self.vmax_input.value = np.max(self.img_vol)
            
            self.on_axis_change(None)
            self.update_plot(None)

    def on_axis_change(self, change):
        if self.img_vol is None: return
        ax = self.axis_dropdown.value
        max_s = self.img_vol.shape[ax] - 1
        self.slice_slider.max = max_s
        self.slice_slider.value = max_s // 2

    def on_window_change(self, change):
        val = self.window_dropdown.value
        if val == "custom":
            self.custom_box.layout.visibility = 'visible'
        elif val == "full":
            self.custom_box.layout.visibility = 'hidden'
            if self.img_vol is not None:
                self.vmin_input.value = np.min(self.img_vol)
                self.vmax_input.value = np.max(self.img_vol)
        else:
            self.custom_box.layout.visibility = 'hidden'
            self.vmin_input.value, self.vmax_input.value = val
        self.update_plot(None)

    def get_slice(self, vol, axis, sl):
        if vol is None: return None
        if axis == 0: return np.rot90(vol[sl, :, :])
        elif axis == 1: return np.rot90(vol[:, sl, :])
        return np.rot90(vol[:, :, sl])

    def update_plot(self, change):
        if self.img_vol is None: return
        ax, sl = self.axis_dropdown.value, self.slice_slider.value
        vmin, vmax = self.vmin_input.value, self.vmax_input.value
        img_slice = self.get_slice(self.img_vol, ax, sl)
        
        with self.output:
            clear_output(wait=True)
            fig, ax_plot = plt.subplots(figsize=(8, 8))
            im = ax_plot.imshow(img_slice, cmap='gray', vmin=vmin, vmax=vmax)
            plt.colorbar(im, ax=ax_plot, fraction=0.046, pad=0.04)
            ax_plot.set_title(f"Slice {sl} | vmin: {vmin:.1f}, vmax: {vmax:.1f}")
            ax_plot.axis('off')
            plt.show()

In [8]:
viewer = SimpleNiftiViewer()

Output()